In [35]:
import os
import re
import pandas as pd
import importlib

# Extracción

In [40]:
import FuncionesLectura as fl
importlib.reload(fl)

df_corpus=fl.leerCorpus(limpiar=False)
df_corpus.head()

,id,speaker,transcription,text,gloss_es,free_translation,morpheme_break,pos,file
0,002,Roberto,entonces estamos grabando estamos acá con Elias,entonces estamos grabando estamos acá con Elias,,entonces estamos grabando estamos acá con Elias,,,EC-cancion.aku-2013.txt
1,003,Roberto,y Elias va a hacer una canción propia de iscon...,y Elias va a hacer una canción propia de iscon...,,y Elias va a hacer una canción propia de iscon...,,,EC-cancion.aku-2013.txt
2,004,Roberto,ya puedes empezar,ya puedes empezar,,ya puedes empezar,,,EC-cancion.aku-2013.txt
3,005,Elias,non kentihokobi,non kentihokobi,1PL =GEN olla -DIM =ENF,nuestra ollita,no =n kenti -hoko =bi,pron =clit. n. -suf =clit.,EC-cancion.aku-2013.txt
4,006,Elias,non kentihokobi,non kentihokobi,1PL =GEN olla -DIM =ENF,nuestra ollita,no =n kenti -hoko =bi,pron =clit. n. -suf =clit.,EC-cancion.aku-2013.txt


In [54]:
import nltk 
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

#Crear un corpus con los textos en df
corpus = PlaintextCorpusReader('../corpus', '.*')
for i, row in df.iterrows():
    #Crear un archivo con el id como nombre y el texto como contenido
    with open(f'../corpus/{row["id"]}.txt', 'w') as file:
        file.write(row['texto'])